In [35]:
import requests
import json
from apikeys import av_key
import pandas as pd
import os
import csv
import sys


In [3]:
base_url = 'https://www.alphavantage.co/query?function='


sector_url = base_url +'SECTOR&apikey='+ av_key


sector_response = requests.get(sector_url)

response_json = sector_response.json()
print(json.dumps(response_json, indent=4, sort_keys=True))



{
    "Meta Data": {
        "Information": "US Sector Performance (realtime & historical)",
        "Last Refreshed": "04:20 PM ET 06/26/2018"
    },
    "Rank A: Real-Time Performance": {
        "Consumer Discretionary": "0.68%",
        "Consumer Staples": "-0.45%",
        "Energy": "1.40%",
        "Financials": "-0.37%",
        "Health Care": "-0.28%",
        "Industrials": "0.33%",
        "Information Technology": "0.48%",
        "Materials": "0.36%",
        "Real Estate": "0.50%",
        "Telecommunication Services": "-0.42%",
        "Utilities": "-0.01%"
    },
    "Rank B: 1 Day Performance": {
        "Consumer Discretionary": "-2.18%",
        "Consumer Staples": "0.44%",
        "Energy": "-2.20%",
        "Financials": "-1.03%",
        "Health Care": "-0.90%",
        "Industrials": "-1.24%",
        "Information Technology": "-2.28%",
        "Materials": "-1.46%",
        "Real Estate": "-0.18%",
        "Telecommunication Services": "-0.08%",
        "Utilitie

In [78]:
symbol = '^GSPC' # ^GSPC is SP 500


time_series_url = base_url +'TIME_SERIES_DAILY&symbol='+symbol+'&outputsize=full&apikey='+ av_key

daily_response = requests.get(time_series_url)
                          
response_json2 = daily_response.json()   

opening = []
closing = []
date = []


for key in response_json2["Time Series (Daily)"]:
    
    opening.append(response_json2["Time Series (Daily)"][key]["1. open"])
    closing.append(response_json2["Time Series (Daily)"][key]["4. close"])
    date.append(key)

sp_df = pd.DataFrame({
    'Date': date,
    'S&P500 Open': opening,
    'S&P500 Close': closing
})


sp_df = sp_df[["Date", "S&P500 Open","S&P500 Close"]]
sp_df = sp_df.sort_values('Date', ascending = True)  

sp_df.head()

,Date,S&P500 Open,S&P500 Close
4649,2000-01-03,1469.2500,1455.2200
4648,2000-01-04,1455.2200,1399.4200
4647,2000-01-05,1399.4200,1402.1100
4646,2000-01-06,1402.1100,1403.4500
4645,2000-01-07,1403.4500,1441.4700


In [81]:
# Loop to calculate

same_day =[]
one_day = []
two_day =[]

for row in range(len(sp_df)):
   same_day_open= float(sp_df["S&P500 Open"][row])
   same_day_close= float(sp_df["S&P500 Close"][row])
   same_day_change = round(((same_day_close - same_day_open)/same_day_open ) * 100 ,2)
   same_day.append(same_day_change)
   
   if row + 1 < len(sp_df):
       prev_day_close= float(sp_df["S&P500 Close"][row])
       next_day_close = float(sp_df["S&P500 Close"][(row + 1)])
       one_day_change = round(((next_day_close - prev_day_close)/prev_day_close ) * 100 ,2)
       one_day.append(one_day_change)
       
   else:
       one_day_change = 0
       one_day.append(one_day_change)

   if row + 2 < len(sp_df):
       prev_day_close= float(sp_df["S&P500 Close"][row])
       
       two_day_close = float(sp_df["S&P500 Close"][(row + 2)])
       two_day_change = round(((two_day_close - prev_day_close)/prev_day_close ) * 100 ,2)
       two_day.append(two_day_change)
       
   else:
       two_day_change = 0
       two_day.append(two_day_change)

   
   
print (same_day[0])
print (same_day[252])

print (one_day[0])
print (one_day[251])

print (two_day[0])
print (two_day[252])

0.03
-0.17
-0.22
0.81
1.17
-0.19


In [82]:
sp_df["Same Day Change"] = same_day
sp_df["One Day Change"] = one_day
sp_df["Two Day Change"] = two_day


sp_df.head()

    
    
    

,Date,S&P500 Open,S&P500 Close,Same Day Change,One Day Change,Two Day Change
4649,2000-01-03,1469.2500,1455.2200,0.03,-0.22,1.17
4648,2000-01-04,1455.2200,1399.4200,-0.94,1.39,1.20
4647,2000-01-05,1399.4200,1402.1100,-0.21,-0.19,0.45
4646,2000-01-06,1402.1100,1403.4500,-0.70,0.64,0.47
4645,2000-01-07,1403.4500,1441.4700,-0.09,-0.17,0.23


2769.28

'2018-06-25'